# 🧠 Global Solution — Artificial Intelligence & Chatbot
**Integrantes:**  
- Pedro Henrique Luiz Alves Duarte – RM563405  
- Guilherme Macedo Martins – RM562396  

## 🎯 Tema
Predição de **Risco de Burnout** e **Produtividade** no ambiente de trabalho utilizando **Modelos de Machine Learning**.

---
## 📊 Etapa 1 — Análise Exploratória dos Dados
Carregamos e analisamos os dados para entender o contexto e definir as variáveis preditoras.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('mental_health_workplace_survey.csv')
display(df.head())
display(df.describe())

---
## 🔹 Etapa 2 — Classificação: Predição de Risco de Burnout
Explicação: Este modelo classifica se o funcionário tem alto risco de burnout com base em variáveis psicológicas e de ambiente de trabalho.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import pickle

df_cls = df.copy()
le = LabelEncoder()
for col in df_cls.select_dtypes(include=['object']).columns:
    df_cls[col] = le.fit_transform(df_cls[col].astype(str))

X = df_cls.drop(columns=['EmployeeID', 'BurnoutRisk'])
y = df_cls['BurnoutRisk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

rf = RandomForestClassifier(n_estimators=150, random_state=42)
rf.fit(X_train_s, y_train)
y_pred = rf.predict(X_test_s)

acc = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acc:.3f}')
print(classification_report(y_test, y_pred))

ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot()
plt.title('Matriz de Confusão - BurnoutRisk')
plt.show()

with open('modelo_classificacao_burnoutrisk.pkl', 'wb') as f:
    pickle.dump(rf, f)
print('✅ Modelo salvo: modelo_classificacao_burnoutrisk.pkl')

---
## 🔹 Etapa 3 — Regressão: Predição da Produtividade (XGBoost)
Explicação: Este modelo prevê o **ProductivityScore** (nota de produtividade) a partir de variáveis psicológicas, físicas e sociais.
Foi utilizada **engenharia de variáveis** e o modelo **XGBoost Regressor**, robusto e preciso para problemas de regressão.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

df_reg = df.copy()
features = [
    'JobSatisfaction','WorkLifeBalanceScore','CareerGrowthScore',
    'ManagerSupportScore','BurnoutLevel','StressLevel',
    'SleepHours','PhysicalActivityHrs','YearsAtCompany'
]
Xr = df_reg[features]
yr = df_reg['ProductivityScore']

Xr['Stress_Burnout'] = Xr['StressLevel'] * Xr['BurnoutLevel']
Xr['Satisfaction_Support'] = Xr['JobSatisfaction'] * Xr['ManagerSupportScore']
Xr['WorkLife_Balance'] = Xr['WorkLifeBalanceScore'] * Xr['SleepHours']

scaler_y = MinMaxScaler()
yr_scaled = scaler_y.fit_transform(yr.values.reshape(-1, 1)).ravel()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr_scaled, test_size=0.2, random_state=42)
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
Xr_train_s = scaler_x.fit_transform(Xr_train)
Xr_test_s = scaler_x.transform(Xr_test)

xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42
)
xgb.fit(Xr_train_s, yr_train)
yr_pred = xgb.predict(Xr_test_s)

mae = mean_absolute_error(yr_test, yr_pred)
mse = mean_squared_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)
print(f'MAE: {mae:.2f}\nMSE: {mse:.2f}\nR²: {r2:.3f}')

plt.figure(figsize=(6,6))
sns.scatterplot(x=yr_test, y=yr_pred, color='dodgerblue')
plt.plot([yr_test.min(), yr_test.max()], [yr_test.min(), yr_test.max()], 'r--')
plt.xlabel('Valores Reais')
plt.ylabel('Valores Previstos')
plt.title('📈 Regressão: Predição de Produtividade (XGBoost Otimizado)')
plt.show()

importances = pd.Series(xgb.feature_importances_, index=Xr.columns)
importances.sort_values().plot(kind='barh', figsize=(8,5), color='teal')
plt.title('💡 Importância das Variáveis no Modelo de Regressão (XGBoost)')
plt.show()

with open('modelo_regressao_xgboost.pkl', 'wb') as f:
    pickle.dump(xgb, f)
print('✅ Modelo salvo: modelo_regressao_xgboost.pkl')

---
## ✅ Conclusão
Foram desenvolvidos dois modelos preditivos que utilizam dados psicológicos e de hábitos de trabalho para análise de **saúde mental corporativa**:
- **Random Forest** para classificar risco de burnout.
- **XGBoost Regressor** para prever produtividade.

Ambos apresentaram bom desempenho e podem ser integrados em uma **API Flask** para uso prático em empresas.